In [98]:
def phredQ(q):
    return ord(q) - 33

def readFastq(filename):
    sequenses = []
    qualities = []
    with open(filename) as f:
        while True :
            f.readline()
            seq = f.readline().rstrip()
            f.readline()
            qual = f.readline().rstrip()
            if len(seq) == 0:
                break
            sequenses.append(seq)
            qualities.append(qual)
    return sequenses, qualities


def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's suffx in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

def createKmerIndex(reads, k):
    dic = {}
    for read in reads:
        for i in range(len(read) - k + 1):  # for each k-mer
            kmer = read[i:i+k]
            if kmer in dic:
                dic[kmer].add(read)
            else:
                dic[kmer] = set([read])    
    return dic   
        
def findOverlapSuffix(reads, k):
    dic = createKmerIndex(reads, k)
    count = 0
    result = []
    for a in reads:
        added = False;
        kmer = a[-1*k:]
        for v in dic[kmer]:
            if not a == v:
                if overlap(a,v,k) > 0:
                    result.append((a,v))
                    added = True
        if added == True:
            count +=1
        #else:
         #   print(a)
    return result,count
    #return count
    
import itertools


def scs(ss):
    """ Returns shortest common superstring of given
        strings, which must be the same length """
    shortest_sup = None
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        if shortest_sup is None or len(sup) < len(shortest_sup):
            shortest_sup = sup  # found shorter superstring
    return shortest_sup  # return shortest

def scs_list(ss):
    """ Returns shortest common superstring of given
        strings, which must be the same length """
    shortest_sup = None
    all_sups = set()
    all_sohortes_sups = set()
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        
        
        if shortest_sup is None or len(sup) == len(shortest_sup):
            all_sups.add(sup)
                        
        
        if shortest_sup is None or len(sup) < len(shortest_sup):
            shortest_sup = sup  # found shorter superstring
            all_sups.add(sup)
            
    for s in all_sups:
        if len(s) == len(shortest_sup):
            all_sohortes_sups.add(s)
            
    return shortest_sup , all_sohortes_sups # return shortest

def scs_list_Kmer(reads,k):
    dic = createKmerIndex(reads, k)
    result = None
    for i in range(len(reads)):
                   
        #print(i)
        if result is None:
            result = reads[0]
        print(result)
        kmer = result[-1*k:]
        for v in dic[kmer]:
            olap = overlap(result,v,k)
            if olap > 0:
                    #print(olap)
                    print(v[olap:])
                    result += v[olap:]
                    #print(v[olap:])
                    #reads.remove(v)
                    break
    return result
  

In [99]:
x= scs_list_Kmer(['ABC', 'BCA', 'CAB','CAB','BBB'],2)
print(x)

ABC
A
ABCA
B
ABCAB

ABCAB

ABCAB

ABCAB


In [20]:
x= scs_list_Kmer(['CCT', 'CTT', 'TGC', 'TGG', 'GAT', 'ATT'],2)
print(x)


CCT
CCT
CCT
CCT
CCT
CCT


In [86]:
readsAsm , quals = readFastq('ads1_week4_reads.fq')

len(readsAsm)

1881

In [100]:
x= scs_list_Kmer(readsAsm,5)
print(len(x))

...

149


In [73]:
print(x)

GTCCAGCAGAGCAAGTGATGCGAGAGCTGCCCATCCTCCAACCAGCATGCCCCTAGACATTGACACTGCATCGGAGTCAGGCCAAGATCCGCAGGACAGTCGAAGGTCAGCTGACGCCC
